In [5]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import ipywidgets as ipw
from traitlets import Unicode

from astropy.table import Table 

from ccdproc import ImageFileCollection

from reducer.image_browser2 import ButtonHolder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
ifc = ImageFileCollection('ims')

In [159]:
def rec_dd():
    """
    A recursive default dict for holding the button groups.
    
    Capy/pasted from 
    
    https://stackoverflow.com/a/19189356/3486425
    
    Thanks, Andrew Clark!
    """
    return defaultdict(rec_dd)

class NestedButtons(ipw.Box):
    def __init__(self, collection, keys=None, **kwd):
        super().__init__(**kwd)
        
        self._table = collection.summary
        if not isinstance(keys, list):
            keys = [keys]
            
        self._key_names = keys
        self._grouped = self._table.group_by(keys)
        self._depth = len(keys)
        self._top = ipw.Accordion()
        self._top_d = rec_dd()
        
    def _make_leaves(self):
        butt_groups = []

        ays = []
        old_keys = []
        for key, v in zip(self._grouped.groups.keys, self._grouped.groups):
            print(f'{key=}')
            print('----->', list(key))
            print(key.colnames[:-1])
            #print([k for k in key[key.colnames[:-1]]])
            #new_keys = [k for k in key[key.colnames[:-1]]]
            #if new_keys == old_keys:
            #     print('moo')
            # else:
            #     print('oink')

            ntt = self._top_d
            for k in list(key)[:-1]:
                ntt = ntt[k]
                
            # old_keys = new_keys
            b = ButtonHolder(descriptions=v['file'])
            a = ipw.Accordion()
            a.children = [b]
            a.titles = [f'{key.colnames[-1]}: {key[-1]} ({len(v)} image{"s" if len(v) > 1 else ""})']
            butt_groups.append(b)
            ntt[key[-1]] = a
            ays.append(a)
            #path = '😃'.join([k for k in key])
            #ays[path] = a
        return butt_groups, ays

    def _make_parent(self, keys, kid):
        if len(keys) == 1:
            key = keys[0]
            if key not in self._top.titles:
                titles = self._top.titles
                self._top.children = list(self._top.children) + [child]
                self._top.titles = list(titles) + [key]
        else:
            pass
            

class ButtonHolder(ipw.VBox):
    selected_button = Unicode(help='Button that was most recently clicked').tag(sync=True)
    def __init__(self, **kwd):
        super().__init__(**kwd)
        self._buttons = {}
        self.button_layout = ipw.Layout(width='100%', 
                                       height='25px',
                                       min_height='25px',
                                       align_content='flex-start',
                                       justify_content='flex-start')
        font_settings = dict(
            font_weight='bold',
            font_size='0.7em'
        )
        self.unselected_button_style = ipw.ButtonStyle(**font_settings)
        self.selected_button_style = ipw.ButtonStyle(
            button_color='CadetBlue',
            **font_settings
        )
        self.layout = ipw.Layout(overflow='hidden scroll')
        
    def set_buttons(self, descriptions):
        buttons = {}
        for desc in descriptions:
            b = ipw.Button(description=desc, 
                           layout=self.button_layout,
                           style=self.unselected_button_style
                          )
            b.on_click(self._make_button_handler())
            buttons[desc] = b
        self._buttons = buttons
        self.children = list(buttons.values())
        
    def _make_button_handler(self):
        def button_handler(button):
            if self.selected_button and (button.description != self.selected_button):
                self._buttons[self.selected_button].style = self.unselected_button_style
            
            self.selected_button = button.description
            button.style = self.selected_button_style
        
        return button_handler

In [145]:
nbuh = NestedButtons(ifc, keys=['filter']) # , 'exptime', 'object'
b, a = nbuh._make_leaves()

key=<Row index=0>
filter
 str2 
------
    gp
-----> ['gp']
[]
key=<Row index=1>
filter
 str2 
------
    ip
-----> ['ip']
[]


In [153]:
def make_accords(top, top_a, key_names):
    kids = []
    titles = []
    print(key_names)
    print(top)
    if len(key_names) > 2:
        for k, v in top.items():
            a = ipw.Accordion()
            make_accords(top[k], a, key_names[1:])
            titles.append(f'{key_names[0]}: {k}')
            kids.append(a)
        top_a.children = kids
        top_a.titles = titles
    else:
        for k, v in top.items():
            kids.append(v)
        print('moo', k, v, top_a)
        top_a.children = list(top.values()) # kids
        top_a.titles = list(top.keys()) #['moo'] * len(top_a.children)
        #a = v
        #a = list(v.values())[0]
        # titles.append('moo')
        # kids.append(a)
    
    
    

In [154]:
def make_accords_og(top, top_a, key_names):
    kids = []
    titles = []
    print(key_names)
    print(top_a)
    
    for k, v in top.items():
        if len(key_names) > 1:
            a = ipw.Accordion()
            make_accords_og(top[k], a, key_names[1:])
            titles.append(f'{key_names[0]}: {k}')
        else:
            print('moo', k, v)
            a = v
            #a = list(v.values())[0]
            titles.append('moo')
        kids.append(a)
    print(f'{titles=}')
    top_a.children = kids
    top_a.titles = titles
    

In [155]:
make_accords_og(nbuh._top_d, nbuh._top, nbuh._key_names)

['filter']
Accordion(children=(Accordion(children=(ButtonHolder(children=(Button(description='TIC272758199-01_20200701_Paul-P-Feder-0.4m_gp_platesolved.fit', layout=Layout(align_content='flex-start', height='25px', justify_content='flex-start', min_height='25px', width='100%'), style=ButtonStyle(font_size='0.7em', font_weight='bold')),), layout=Layout(overflow='hidden scroll')),), titles=('filter: gp (1 image)',)), Accordion(children=(ButtonHolder(children=(Button(description='TIC106224627.01_20220713_Paul-P-Feder-0.4m_ip_platesolved.fit', layout=Layout(align_content='flex-start', height='25px', justify_content='flex-start', min_height='25px', width='100%'), style=ButtonStyle(font_size='0.7em', font_weight='bold')), Button(description='TIC272758199-01_20200701_Paul-P-Feder-0.4m_ip_platesolved.fit', layout=Layout(align_content='flex-start', height='25px', justify_content='flex-start', min_height='25px', width='100%'), style=ButtonStyle(font_size='0.7em', font_weight='bold')), Button(des

In [156]:
fake_top = ipw.Accordion()

In [157]:
make_accords(nbuh._top_d, fake_top, nbuh._key_names)

['filter']
defaultdict(<function rec_dd at 0x1867863b0>, {'gp': Accordion(children=(ButtonHolder(children=(Button(description='TIC272758199-01_20200701_Paul-P-Feder-0.4m_gp_platesolved.fit', layout=Layout(align_content='flex-start', height='25px', justify_content='flex-start', min_height='25px', width='100%'), style=ButtonStyle(font_size='0.7em', font_weight='bold')),), layout=Layout(overflow='hidden scroll')),), titles=('filter: gp (1 image)',)), 'ip': Accordion(children=(ButtonHolder(children=(Button(description='TIC106224627.01_20220713_Paul-P-Feder-0.4m_ip_platesolved.fit', layout=Layout(align_content='flex-start', height='25px', justify_content='flex-start', min_height='25px', width='100%'), style=ButtonStyle(font_size='0.7em', font_weight='bold')), Button(description='TIC272758199-01_20200701_Paul-P-Feder-0.4m_ip_platesolved.fit', layout=Layout(align_content='flex-start', height='25px', justify_content='flex-start', min_height='25px', width='100%'), style=ButtonStyle(font_size='0

In [158]:
fake_top

Accordion(children=(Accordion(children=(ButtonHolder(children=(Button(description='TIC272758199-01_20200701_Pa…

In [152]:
nbuh._top

Accordion(children=(Accordion(children=(ButtonHolder(children=(Button(description='TIC272758199-01_20200701_Pa…

In [21]:
a[0]

Accordion(children=(ButtonHolder(children=(Button(description='TIC272758199-01_20200701_Paul-P-Feder-0.4m_gp_p…

In [39]:
list(nbuh._top_d.keys())

['gp', 'ip']

In [339]:
ks = nbuh._grouped.groups.keys
columns = ks.colnames

def add_ds(tapas, keyset):
    pass
    

In [340]:
from collections import defaultdict

In [345]:
d = defaultdict(dict)
def rec_dd():
    return defaultdict(rec_dd)

In [346]:
top = rec_dd()

for key in ks:
    ntt = top
    for k in key:
        ntt = ntt[k]

In [352]:
top

defaultdict(<function __main__.rec_dd()>,
            {'gp': defaultdict(<function __main__.rec_dd()>,
                         {120.0: defaultdict(<function __main__.rec_dd()>,
                                      {'tic-272758199': defaultdict(<function __main__.rec_dd()>,
                                                   {})})}),
             'ip': defaultdict(<function __main__.rec_dd()>,
                         {120.0: defaultdict(<function __main__.rec_dd()>,
                                      {'TIC-106224627': defaultdict(<function __main__.rec_dd()>,
                                                   {}),
                                       'TIC-402828941': defaultdict(<function __main__.rec_dd()>,
                                                   {}),
                                       'tic-272758199': defaultdict(<function __main__.rec_dd()>,
                                                   {})})})})

In [233]:
buh = ButtonHolder()
buh.layout.width = "20%"
buh.layout.height = "200px"
buh.layout.border = 'solid'
butts = [str(i) for i in range(40)]
buh.set_buttons(butts) # ['asaodjfoerjfoirjfoierf'] * 30)

In [234]:
ab = ipw.HBox()
ab.layout.border = 'solid'
ab

HBox(layout=Layout(border_bottom='solid', border_left='solid', border_right='solid', border_top='solid'))

In [235]:
ab.children = (buh,)

In [ ]:
buh.selected_button

In [ ]:
pooh = ipw.Select(options=['a', 'b', 'c'])

pooh

In [ ]:
pooh.value = None

In [247]:
d = dict(a=[0,0,4,4,5], b=[1,1,1,2,3], c=range(5))
tab = Table(d)

In [248]:
for c in tab.columns:
    print(c)

a
b
c


In [249]:
tab.columns, tab.colnames

(<TableColumns names=('a','b','c')>, ['a', 'b', 'c'])

In [278]:
tab.group_by(['a', 'b']).groups.keys[0].columns

<TableColumns names=('a','b')>